In [30]:
import datetime
import json

import boto3
import requests

In [31]:
USERNAME = "szbeni"

In [32]:
DATE_PARAM = "2025-11-03"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/2025/11/03
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"03","articles":[{"article":"Main_Page","views":6058900,"rank":1},{"article":"Special:Search","views":977237,"rank":2},{"article":"Google_Chrome","views":357501,"rank":3},{"article":".xxx","views":340519,"rank":4},{"article":"Women's_Cricket_World_Cup","views":270596,"rank":5},{"article":"Wikipedia:Featured_pictures","views":261727,"rank":6},{"article":"Diane_Ladd","views":259554,"rank":7},{"article":"Zohr...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 56552


In [33]:
wiki_response_parsed = wiki_server_response.json()
most_viewed = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in most_viewed[:5]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(most_viewed)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6058900, "rank": 1, "date": "2025-11-03", "retrieved_at": "2025-12-10T17:29:01.241389"}
{"title": "Special:Search", "views": 977237, "rank": 2, "date": "2025-11-03", "retrieved_at": "2025-12-10T17:29:01.241389"}
{"title": "Google_Chrome", "views": 357501, "rank": 3, "date": "2025-11-03", "retrieved_at": "2025-12-10T17:29:01.241389"}
{"title": ".xxx", "views": 340519, "rank": 4, "date": "2025-11-03", "retrieved_at": "2025-12-10T17:29:01.241389"}
{"title": "Women's_...


In [34]:
S3_WIKI_BUCKET = "szbeni-wikidata"
s3 = boto3.client("s3")

assert USERNAME != "szbeni-wikidata>", "Please set your USERNAME at the top of the notebook"
assert S3_WIKI_BUCKET.endswith("-wikidata"), "Bucket name must end with '-wikidata'"

try:
    s3.head_bucket(Bucket=S3_WIKI_BUCKET)
    print(f"Bucket {S3_WIKI_BUCKET} exists!")
except Exception as e:
    print(f"Bucket {S3_WIKI_BUCKET} not found: {e}")
    raise

Bucket szbeni-wikidata exists!


In [35]:
s3_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=s3_key,
    Body=json_lines,
)
print(f"Uploaded {len(most_viewed)} records to s3://{S3_WIKI_BUCKET}/{s3_key}")

Uploaded 1000 records to s3://szbeni-wikidata/raw-views/raw-views-2025-11-03.json
